In [ ]:
# ============================================ 
# STEP 3 — Build or Load Hybrid (Semantic + Lexical) Retriever (RAG-Version 6)
# ============================================

from langchain_community.vectorstores import LanceDB
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import lancedb
import torch
import time
from tqdm import tqdm
from pathlib import Path

# --- Parameters ---
TOP_K = 8
EMBED_MODEL = "mixedbread-ai/mxbai-embed-large-v1"  # Dense semantic embeddings
INDEX_DIR = Path("data/index/lancedb_v6_hybrid")

# --- Auto-detect device ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"💻 Using device: {DEVICE.upper()}")

# --- Auto-adjust batch size by VRAM ---
def auto_batch_size():
    if DEVICE != "cuda":
        return 16
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    if vram_gb >= 22:
        return 128
    elif vram_gb >= 12:
        return 64
    else:
        return 32

BATCH_SIZE = auto_batch_size()
print(f"🧠 GPU VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"⚙️ Batch size set to: {BATCH_SIZE}")

# --- Initialize dense embedding model ---
embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs={"device": DEVICE},
    encode_kwargs={"batch_size": BATCH_SIZE}
)

# ------------------------------------------------------------
# Function: Build or Load LanceDB Semantic Index
# ------------------------------------------------------------
def build_or_load_lancedb_index(index_dir=INDEX_DIR, chunks=None):
    """
    Builds or loads a LanceDB semantic index for hybrid dense + lexical retrieval.
    """
    index_dir.mkdir(parents=True, exist_ok=True)
    db = lancedb.connect(str(index_dir))

    # --- Load existing index if available ---
    if "documents" in db.table_names():
        print("📦 Existing LanceDB semantic index found. Loading from disk...")
        table = db.open_table("documents")
        vectorstore = LanceDB(connection=db, table=table, embedding=embeddings)
        print("✅ LanceDB semantic index loaded successfully.")
        return vectorstore

    # --- Validate chunks ---
    if not chunks:
        raise RuntimeError("❌ No chunks provided. Please run Step 2 first.")

    total = len(chunks)
    print(f"🧱 Building new LanceDB semantic index on {DEVICE.upper()}...")
    print(f"📊 Total chunks to embed: {total:,}")

    start_time = time.time()

    # --- Create LanceDB vector store from documents ---
    vectorstore = LanceDB.from_documents(
        tqdm(chunks, desc="🔢 Embedding text chunks", ncols=100),
        embedding=embeddings,
        uri=str(index_dir),
        table_name="documents"
    )

    duration = time.time() - start_time
    print(f"💾 Index saved to {index_dir}")
    print(f"⏱️ Build completed in {duration/60:.2f} min ({duration:.1f} sec)")
    return vectorstore


# ------------------------------------------------------------
# Execute step — Build or Load Index
# ------------------------------------------------------------
vectorstore = build_or_load_lancedb_index(INDEX_DIR, chunks)

# ------------------------------------------------------------
# Step 3.1 — Create Semantic and Lexical Retrievers
# ------------------------------------------------------------
print("⚙️ Initializing hybrid retrievers (semantic + lexical)...")
semantic_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": TOP_K}
)

keyword_retriever = BM25Retriever.from_documents(chunks)

# ------------------------------------------------------------
# Step 3.2 — Combine Dense + Sparse Retrieval (Weighted Ensemble)
# ------------------------------------------------------------
retriever = EnsembleRetriever(
    retrievers=[semantic_retriever, keyword_retriever],
    weights=[0.7, 0.3]  # αsemantic = 0.7, αkeyword = 0.3
)

# ------------------------------------------------------------
# Final Summary
# ------------------------------------------------------------
print(f"✅ Hybrid Retriever ready (LanceDB + BM25, αsemantic=0.7, αkeyword=0.3, top_k={TOP_K})")
print("📈 Configuration Summary:")
print(f"   • Embedding Model: {EMBED_MODEL}")
print(f"   • Vector DB: LanceDB (semantic)")
print(f"   • Lexical Model: BM25 (keyword)")
print(f"   • Retrieval Mode: Hybrid Dense + Sparse Fusion")
print(f"   • Top-k: {TOP_K}")
print(f"   • Batch Size: {BATCH_SIZE}")
print(f"   • Device: {DEVICE.upper()}")


In [ ]:
# ============================================ 
# STEP 3 — Build or Load Multi-Embedding Ensemble Index (RAG-Version 7)
# ============================================

from langchain_community.vectorstores import LanceDB
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import EnsembleRetriever
import lancedb
import torch
import time
from tqdm import tqdm
from pathlib import Path

# --- Parameters ---
TOP_K = 8
INDEX_DIR = Path("data/index/lancedb_v7_ensemble")

# Embedding models (semantic, hybrid, instruction-aware)
EMBED_MODELS = {
    "semantic": "sentence-transformers/all-mpnet-base-v2",
    "hybrid": "mixedbread-ai/mxbai-embed-large-v1",
    "instruction": "hkunlp/instructor-large"
}

# --- Auto-detect device ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"💻 Using device: {DEVICE.upper()}")

# --- Auto-adjust batch size based on VRAM ---
def auto_batch_size():
    if DEVICE != "cuda":
        return 16
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    if vram_gb >= 22:
        return 128
    elif vram_gb >= 12:
        return 64
    else:
        return 32

BATCH_SIZE = auto_batch_size()
print(f"🧠 GPU VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"⚙️ Batch size set to: {BATCH_SIZE}")

# ------------------------------------------------------------
# Function: Build or Load a LanceDB Index for each embedding model
# ------------------------------------------------------------
def build_or_load_lancedb_index(model_name: str, model_label: str, chunks, base_dir=INDEX_DIR):
    """
    Builds or loads a LanceDB index for a specific embedding model.
    Each model stores its own embeddings to allow ensemble fusion later.
    """
    index_path = base_dir / model_label
    index_path.mkdir(parents=True, exist_ok=True)
    db = lancedb.connect(str(index_path))

    # --- Initialize embedding model ---
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": DEVICE},
        encode_kwargs={"batch_size": BATCH_SIZE}
    )

    # --- Load existing index if available ---
    if "documents" in db.table_names():
        print(f"📦 Existing LanceDB index ({model_label}) found – loading...")
        table = db.open_table("documents")
        return LanceDB(connection=db, table=table, embedding=embeddings)

    # --- Validate chunks ---
    if not chunks:
        raise RuntimeError("❌ No chunks provided. Please run Step 2 first.")

    total = len(chunks)
    print(f"🧱 Building LanceDB index for '{model_label}' on {DEVICE.upper()}...")
    print(f"📊 Total chunks to embed: {total:,}")

    start_time = time.time()
    vectorstore = LanceDB.from_documents(
        tqdm(chunks, desc=f"🔢 Embedding chunks ({model_label})", ncols=100),
        embedding=embeddings,
        uri=str(index_path),
        table_name="documents",
    )

    duration = time.time() - start_time
    print(f"💾 Saved LanceDB index ({model_label}) → {index_path}")
    print(f"⏱️ Build completed in {duration/60:.2f} min ({duration:.1f} sec)")
    return vectorstore


# ------------------------------------------------------------
# Step 3.1 – Build or Load all VectorStores
# ------------------------------------------------------------
vectorstores = {}
for label, model_name in EMBED_MODELS.items():
    vectorstores[label] = build_or_load_lancedb_index(model_name, label, chunks)

# ------------------------------------------------------------
# Step 3.2 – Create Individual Retrievers
# ------------------------------------------------------------
retrievers = {
    label: store.as_retriever(search_type="similarity", search_kwargs={"k": TOP_K})
    for label, store in vectorstores.items()
}

# ------------------------------------------------------------
# Step 3.3 – Combine via Ensemble Retriever
# ------------------------------------------------------------
retriever = EnsembleRetriever(
    retrievers=list(retrievers.values()),
    weights=[0.4, 0.3, 0.3]  # semantic = 0.4, hybrid = 0.3, instruction = 0.3
)

# ------------------------------------------------------------
# Summary
# ------------------------------------------------------------
print(f"✅ Retriever ready (Multi-Embedding Ensemble – semantic=0.4, hybrid=0.3, instruction=0.3, top_k={TOP_K})")
print("📈 Configuration Summary:")
print(f"   • Vector DB: LanceDB (3 indexes)")
print(f"   • Embedding Models: {', '.join(EMBED_MODELS.values())}")
print(f"   • Fusion Weights: [0.4, 0.3, 0.3]")
print(f"   • Batch Size: {BATCH_SIZE}")
print(f"   • Device: {DEVICE.upper()}")


In [ ]:
# ============================================ 
# STEP 3 — Build or Load LanceDB Index with Dynamic Chunk Fusion (RAG-Version 8)
# ============================================

from langchain_community.vectorstores import LanceDB
from langchain.embeddings import HuggingFaceEmbeddings
import lancedb
import torch
import time
from tqdm import tqdm
from pathlib import Path

# --- Parameters ---
TOP_K = 6  # fewer but richer context chunks
EMBED_MODEL = "mixedbread-ai/mxbai-embed-large-v1"
INDEX_DIR = Path("data/index/lancedb_v8_chunkfusion")

# --- Device setup ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"💻 Using device: {DEVICE.upper()}")

# --- Auto-adjust batch size based on VRAM ---
def auto_batch_size():
    if DEVICE != "cuda":
        return 16
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    if vram_gb >= 22:
        return 128
    elif vram_gb >= 12:
        return 64
    else:
        return 32

BATCH_SIZE = auto_batch_size()
print(f"🧠 GPU VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"⚙️ Batch size set to: {BATCH_SIZE}")

# ------------------------------------------------------------
# Function: Dynamic Chunk Fusion
# ------------------------------------------------------------
def fuse_adjacent_chunks(chunks, overlap_threshold=0.5, max_length=2000):
    """
    Dynamically merges consecutive text chunks that show high overlap or continuity.
    Produces longer, semantically coherent chunks for richer retrieval.
    """
    fused_chunks = []
    buffer = None

    for c in chunks:
        text = c.page_content.strip()
        if not text:
            continue

        if buffer is None:
            buffer = c
            continue

        # Token-overlap ratio
        buf_words = set(buffer.page_content.split())
        new_words = set(text.split())
        overlap = len(buf_words & new_words) / max(len(buf_words), 1)

        # Merge if overlap high or current chunk still short
        if overlap > overlap_threshold or len(buffer.page_content) < max_length:
            buffer.page_content += " " + text
        else:
            fused_chunks.append(buffer)
            buffer = c

    if buffer:
        fused_chunks.append(buffer)

    print(f"🧩 Fused {len(chunks)} → {len(fused_chunks)} chunks (context-optimized).")
    return fused_chunks


# ------------------------------------------------------------
# Function: Build or Load LanceDB Index
# ------------------------------------------------------------
def build_or_load_lancedb_index(index_dir=INDEX_DIR, chunks=None):
    """
    Builds or loads a LanceDB index from dynamically fused text chunks.
    Generates longer, context-aware embeddings for improved coherence.
    """
    index_dir.mkdir(parents=True, exist_ok=True)
    db = lancedb.connect(str(index_dir))

    # --- Load existing index ---
    if "documents" in db.table_names():
        print("📦 Existing LanceDB (Chunk-Fused) index found – loading...")
        table = db.open_table("documents")
        vectorstore = LanceDB(connection=db, table=table, embedding=embeddings)
        print("✅ LanceDB (Chunk-Fused) index loaded successfully.")
        return vectorstore

    if not chunks:
        raise RuntimeError("❌ No chunks provided. Please run Step 2 first.")

    # --- Apply fusion ---
    fused_chunks = fuse_adjacent_chunks(chunks, overlap_threshold=0.5, max_length=2000)

    print(f"🧱 Building LanceDB (Chunk-Fused) index on {DEVICE.upper()}...")
    print(f"📊 Total fused chunks to embed: {len(fused_chunks):,}")
    start_time = time.time()

    vectorstore = LanceDB.from_documents(
        tqdm(fused_chunks, desc="🔢 Embedding fused text chunks", ncols=100),
        embedding=embeddings,
        uri=str(index_dir),
        table_name="documents",
    )

    duration = time.time() - start_time
    print(f"💾 Saved LanceDB (Chunk-Fused) index → {index_dir}")
    print(f"⏱️ Build completed in {duration/60:.2f} min ({duration:.1f} sec)")
    return vectorstore


# ------------------------------------------------------------
# Step 3 Execution
# ------------------------------------------------------------
embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs={"device": DEVICE},
    encode_kwargs={"batch_size": BATCH_SIZE}
)

vectorstore = build_or_load_lancedb_index(INDEX_DIR, chunks)

# --- Configure retriever ---
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": TOP_K}
)

print(f"✅ Retriever ready (LanceDB + Chunk Fusion + {DEVICE.upper()}, top_k={TOP_K})")
print("📈 Configuration Summary:")
print(f"   • Embedding Model: {EMBED_MODEL}")
print(f"   • Chunk Fusion: enabled (dynamic overlap > 0.5)")
print(f"   • Vector DB: LanceDB")
print(f"   • Top-k: {TOP_K}")
print(f"   • Batch Size: {BATCH_SIZE}")
print(f"   • Device: {DEVICE.upper()}")


In [ ]:
# ============================================ 
# STEP 3 — Speed & Scalability Benchmark (RAG-Version 9)
# ============================================

import torch
import time
import psutil
import random
import numpy as np
from tqdm import tqdm
from pathlib import Path
from langchain_community.vectorstores import LanceDB, FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import lancedb
try:
    from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection
    MILVUS_AVAILABLE = True
except ImportError:
    MILVUS_AVAILABLE = False
    print("⚠️ Milvus not installed — skipping that backend.")

# --- Parameters ---
TOP_K = 8
EMBED_MODEL = "mixedbread-ai/mxbai-embed-large-v1"
INDEX_DIR = Path("data/index/v9_benchmark")
RESULTS_PATH = Path("data/results/v9_benchmark_results.csv")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"💻 Using device: {DEVICE.upper()}")

# --- Adaptive batch size ---
def auto_batch_size():
    if DEVICE != "cuda": return 16
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    return 128 if vram_gb >= 22 else (64 if vram_gb >= 12 else 32)

BATCH_SIZE = auto_batch_size()
print(f"🧠 VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB | Batch Size = {BATCH_SIZE}")

# --- Initialize embedding model ---
embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs={"device": DEVICE},
    encode_kwargs={"batch_size": BATCH_SIZE}
)

# ------------------------------------------------------------
# Helper: measure recall@k
# ------------------------------------------------------------
def recall_at_k(retriever, queries, gold_answers, k=TOP_K):
    correct = 0
    for q, gold in zip(queries, gold_answers):
        results = retriever.get_relevant_documents(q)
        texts = [r.page_content for r in results]
        if any(g in " ".join(texts) for g in gold.split()):
            correct += 1
    return correct / len(queries)

# ------------------------------------------------------------
# Benchmark LanceDB
# ------------------------------------------------------------
def benchmark_lancedb(chunks):
    print("\n🚀 Benchmarking LanceDB ...")
    db_path = INDEX_DIR / "lancedb"
    db_path.mkdir(parents=True, exist_ok=True)
    db = lancedb.connect(str(db_path))
    start = time.time()
    vectorstore = LanceDB.from_documents(
        tqdm(chunks, desc="🔹 Embedding (LanceDB)", ncols=90),
        embedding=embeddings,
        uri=str(db_path),
        table_name="documents"
    )
    build_time = time.time() - start
    retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})
    q_sample = [c.page_content[:200] for c in random.sample(chunks, min(20, len(chunks)))]
    start = time.time()
    for q in q_sample:
        retriever.get_relevant_documents(q)
    query_time = (time.time() - start) / len(q_sample)
    mem = psutil.virtual_memory().used / 1e9
    return {"DB":"LanceDB","Build_s":build_time,"Query_s":query_time,"Mem_GB":mem}

# ------------------------------------------------------------
# Benchmark FAISS
# ------------------------------------------------------------
def benchmark_faiss(chunks):
    print("\n🚀 Benchmarking FAISS (HNSW) ...")
    start = time.time()
    vectorstore = FAISS.from_documents(chunks, embedding=embeddings)
    build_time = time.time() - start
    retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})
    q_sample = [c.page_content[:200] for c in random.sample(chunks, min(20, len(chunks)))]
    start = time.time()
    for q in q_sample:
        retriever.get_relevant_documents(q)
    query_time = (time.time() - start) / len(q_sample)
    mem = psutil.virtual_memory().used / 1e9
    return {"DB":"FAISS","Build_s":build_time,"Query_s":query_time,"Mem_GB":mem}

# ------------------------------------------------------------
# Benchmark Milvus (optional)
# ------------------------------------------------------------
def benchmark_milvus(chunks):
    if not MILVUS_AVAILABLE:
        return None
    print("\n🚀 Benchmarking Milvus ...")
    connections.connect("default", host="localhost", port="19530")
    if utility.has_collection("rag_v9"): utility.drop_collection("rag_v9")
    schema = CollectionSchema([
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=len(embeddings.embed_query("test"))),
    ], "RAG V9 Benchmark")
    collection = Collection("rag_v9", schema)
    start = time.time()
    vecs = [embeddings.embed_query(c.page_content) for c in tqdm(chunks[:5000], desc="🔹 Milvus embedding")]
    collection.insert([list(range(len(vecs))), vecs])
    collection.create_index("embedding", {"index_type":"HNSW","metric_type":"L2"})
    build_time = time.time() - start
    collection.load()
    q_sample = [c.page_content[:200] for c in random.sample(chunks, min(20, len(chunks)))]
    start = time.time()
    for q in q_sample:
        qv = embeddings.embed_query(q)
        collection.search([qv],"embedding",param={"metric_type":"L2","params":{"ef":32}},limit=TOP_K)
    query_time = (time.time() - start) / len(q_sample)
    mem = psutil.virtual_memory().used / 1e9
    return {"DB":"Milvus","Build_s":build_time,"Query_s":query_time,"Mem_GB":mem}

# ------------------------------------------------------------
# Execute benchmark
# ------------------------------------------------------------
def run_benchmark(chunks):
    results = []
    results.append(benchmark_lancedb(chunks))
    results.append(benchmark_faiss(chunks))
    if MILVUS_AVAILABLE:
        milvus_res = benchmark_milvus(chunks)
        if milvus_res: results.append(milvus_res)
    print("\n📊 Benchmark Results:")
    for r in results:
        print(f" {r['DB']:8s} | Build {r['Build_s']/60:.2f} min | Query {r['Query_s']*1000:.2f} ms | Mem {r['Mem_GB']:.1f} GB")
    import pandas as pd
    df = pd.DataFrame(results)
    RESULTS_PATH.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(RESULTS_PATH, index=False)
    print(f"💾 Saved results → {RESULTS_PATH}")
    return df

# ------------------------------------------------------------
# Run
# ------------------------------------------------------------
df_results = run_benchmark(chunks)
print("\n✅ RAG-Version 9 Speed & Scalability Benchmark completed.")
